In [1]:
from lib import * # wrime data is loaded here.

0        0
1        1
2        0
3        0
4        1
        ..
43195    1
43196    2
43197    2
43198    0
43199    2
Name: Avg. Readers_Joy, Length: 43200, dtype: int64
0        [0, 2, 0, 0, 0, 0, 0, 0]
1        [1, 0, 0, 2, 0, 0, 0, 0]
2        [0, 0, 0, 1, 0, 0, 0, 0]
3        [0, 1, 0, 0, 0, 0, 1, 0]
4        [1, 0, 0, 1, 0, 0, 0, 0]
                   ...           
43195    [1, 0, 0, 1, 0, 0, 0, 0]
43196    [2, 0, 2, 0, 0, 0, 0, 0]
43197    [2, 0, 0, 0, 0, 0, 0, 0]
43198    [0, 0, 2, 0, 0, 0, 0, 0]
43199    [2, 0, 0, 0, 0, 0, 0, 1]
Name: readers_emotion_intensities, Length: 43200, dtype: object
---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18237 entries, 0 to 18236
Data columns (total 45 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Sentence                     18237 non-null  object
 1   UserID                       18237 non-null  int64 
 2   Datetime                     18237 non-null

In [2]:
# import pandas as pd
# df_wrime = pd.read_table('wrime-ver1.tsv')
# df_wrime.info()

In [3]:
# # Plutchik's eight primary emotions.
# emotion_names = ['Joy', 'Sadness', 'Anticipation', 'Surprise', 'Anger', 'Fear', 'Disgust', 'Trust']

# # Make a new column by making a list of mean over readers（"Avg. Readers_*"） 
# df_wrime['readers_emotion_intensities'] = df_wrime.apply(lambda x: [x['Avg. Readers_' + name] for name in emotion_names], axis=1)


In [4]:
# print(df_wrime.columns)
# print(len(df_wrime.columns))

In [5]:
# # Tweets such that the maximum intensity of the eight primary emotions is more than 2 are selected.
# is_target = df_wrime['readers_emotion_intensities'].map(lambda x: max(x) >= 2)
# df_wrime_target = df_wrime[is_target]


In [2]:
print(df_wrime)

                                                Sentence  UserID  \
0                         ぼけっとしてたらこんな時間｡チャリあるから食べにでたいのに…       1   
1      今日の月も白くて明るい。昨日より雲が少なくてキレイな? と立ち止まる帰り道｡チャリなし生活も...       1   
2                     早寝するつもりが飲み物がなくなりコンビニへ｡ん､今日、風が涼しいな。       1   
3                                               眠い、眠れない。       1   
4        ただいま? って新体操してるやん!外食する気満々で家に何もないのに!テレビから離れられない…!       1   
...                                                  ...     ...   
43195  真夜中にふと思い立ち、ノートPCを持って部屋を出て、ダイニングで仕事したらすんごい捗った。\...      80   
43196  ぐっどこんでぃしょん。\n心も頭もクリア。\n秋分の日のおかげかな？\n人と自然としっとり過...      80   
43197  朝から免許の更新へ。\n90分で終わり、出口へ向かうと献血の呼びかけが。\nみんな通り過ぎて...      80   
43198  夜も更けて参りましたが、食後のコーヒーが飲みたいのでドリップ開始…\n\nぼんやり秋の夜長を...      80   
43199  コーヒー休憩（kahavitauko）\n\nいつもの豆なのにすごく美味しくできた \n\n...      80   

               Datetime Train/Dev/Test  Writer_Joy  Writer_Sadness  \
0      2012/07/31 23:48          train           0               1   
1      2012/08/02 23:09          train     

In [2]:
# devide the whole dataset into train / test
df_groups = df_wrime_target.groupby('Train/Dev/Test')
df_train = df_groups.get_group('train')
df_test = pd.concat([df_groups.get_group('dev'), df_groups.get_group('test')])
print('train :', len(df_train))  # train : 17104
print('test :', len(df_test))    # test : 1133


train : 17104
test : 1133


In [8]:
# Install HuggingFace Transformers 
# - transformers 
# - datasets 
# cf. https://huggingface.co/docs/transformers/installation
# ! pip install transformers datasets

# Install fugashi and ipadic 
# ! pip install fugashi ipadic


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# load a pre-trained model
checkpoint = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=8)


/home/om/.miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import numpy as np
from datasets import Dataset

# 1.format change: pandas.DataFrame -> datasets.Dataset
target_columns = ['Sentence', 'readers_emotion_intensities']
train_dataset = Dataset.from_pandas(df_train[target_columns])
test_dataset = Dataset.from_pandas(df_test[target_columns])

# 2. apply Tokenizer
def tokenize_function(batch):
    """Tokenizerを適用 （感情強度の正規化も同時に実施する）."""
    tokenized_batch = tokenizer(batch['Sentence'], truncation=True, padding='max_length')
    tokenized_batch['labels'] = [x / np.sum(x) for x in batch['readers_emotion_intensities']]  # normalized
    return tokenized_batch

train_tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
test_tokenized_dataset = test_dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 1133/1133 [00:00<00:00, 5342.86 examples/s]


In [5]:
from transformers import TrainingArguments, Trainer
from datasets import load_metric

# Evaluation Metrics
# https://huggingface.co/docs/transformers/training
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    label_ids = np.argmax(labels, axis=-1)
    return metric.compute(predictions=predictions, references=label_ids)

# Training arguments
# https://huggingface.co/docs/transformers/v4.21.1/en/main_classes/trainer#transformers.TrainingArguments
training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=8,
    num_train_epochs=1.0,
    evaluation_strategy="steps", eval_steps=200)  # evaluate the test data at every 200 steps. 

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=test_tokenized_dataset,
    compute_metrics=compute_metrics,
)

# Training
trainer.train()
# Evaluation
print(trainer.evaluate())

### print(trainer.metrics)

/tmp/ipykernel_11972/617072238.py:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: osamu-maruyama-9999 (maruyama-lab-design). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy
200,No log,0.277081,0.581642
400,No log,0.249603,0.622242
600,0.304000,0.235353,0.664607
800,0.304000,0.224553,0.706973
1000,0.263200,0.214347,0.728155
1200,0.263200,0.219932,0.718447
1400,0.263200,0.211464,0.748455
1600,0.248500,0.208695,0.737864
1800,0.248500,0.206068,0.753751
2000,0.240900,0.205352,0.763460


{'eval_loss': 0.20462995767593384, 'eval_accuracy': 0.7616946160635482, 'eval_runtime': 712.5019, 'eval_samples_per_second': 1.59, 'eval_steps_per_second': 0.199, 'epoch': 1.0}


AttributeError: 'Trainer' object has no attribute 'metrics'

wandb: Network error (ConnectionError), entering retry loop.


In [ ]:
import os
_dir = os.path.join("output", "trained_model")
if not os.path.isdir(_dir):
    os.makedirs(_dir)
trainer.save_model(_dir)

In [ ]:
emotion_names_jp = ['喜び', '悲しみ', '期待', '驚き', '怒り', '恐れ', '嫌悪', '信頼'] 
import matplotlib.pyplot as plt
import seaborn as sns

# sns.set(font='IPAexGothic')

import matplotlib
from matplotlib import font_manager

font_manager.fontManager.addfont("/usr/share/fonts/opentype/ipaexfont-gothic/ipaexg.ttf")
matplotlib.rc('font', family="IPAexGothic")


# Softmax function
# https://www.delftstack.com/ja/howto/numpy/numpy-softmax/
def np_softmax(x):
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x

def analyze_emotion(text, show_fig=False):
    model.eval()

    tokens = tokenizer(text, truncation=True, return_tensors="pt")
    tokens.to(model.device)
    preds = model(**tokens)
    prob = np_softmax(preds.logits.cpu().detach().numpy()[0])
    out_dict = {n: p for n, p in zip(emotion_names_jp, prob)}

    if show_fig:
        plt.figure(figsize=(8, 3))
        df = pd.DataFrame(out_dict.items(), columns=['name', 'prob'])
        sns.barplot(x='name', y='prob', data=df)
        plt.title('Input sentence: ' + text, fontsize=15)
    else:
        print(out_dict)




In [ ]:
analyze_emotion('今日から長期休暇だぁーーー！！！', show_fig=True)

In [ ]:
analyze_emotion('この書類にはコーヒーかかってなくて良かった…。不幸中の幸いだ。', show_fig=True)

In [ ]:

analyze_emotion('なんで自分だけこんな目に遭うんだ……', show_fig=True)

In [ ]:
analyze_emotion('君ならきっとやってくれると思っていたよ！', show_fig=True)

In [ ]:
analyze_emotion('え、今日って休校だったの？', show_fig=True)

In [ ]:
analyze_emotion('明日のプレゼンうまくできるかなぁ…', show_fig=True)

In [ ]:
analyze_emotion('あぁー、イライラするっ！！', show_fig=True)